# Application benchmarks: single node performance comparison

Comparison the single-node performance of three different application benchmarks across a variety of architectures.

## Import required modules for results analysis

In [1]:
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (12,6)
import seaborn as sns
sns.set_style("white", {"font.family": "serif"})
from scipy import stats

In [2]:
import sys
sys.path.append('../python-modules')

## Peak floating point performance

In this section we calculate the peak floating point performance for each of the systems. CPU FLOPS are computed as
(number of cores used) * (single precision FLOPS per cycle) * (frequency). See below for values used for the different systems. GPU FLOPS are computed as (number of GPUs used) * (GPU single precision FLOPS reference value
[ref](https://www.nvidia.com/en-us/data-center/tesla-p100/)).

In [3]:
peak_fp = {}
# Calculate CPU SP FP performance (GFLOP/s): cores * ops/cycle * freq.
peak_fp['ARCHER'] = 24 * 16 * 2.7
peak_fp['Peta4-Skylake'] = 32* 64 * 2.6
peak_fp['Cirrus'] = 36 * 32 * 2.1
peak_fp['Isambard'] = 64 * 16 * 2.2
peak_fp['Tesseract'] = 24 * 32 * 2.1
# Calculate CPU + GPU SP FP performance (GFLOP/s): (cores * ops/cycle * freq.) + (gpu * SP perf.)
peak_fp['Wilkes2-GPU'] = (12 * 32 * 2.2) + (4 * 9300)
peak_fp['JADE']  = (5 * 32 * 2.2) + (1 * 10600)

## Peak memory performance
We define the peak memory performance both in terms of the peak bandwidth measured using the StarSTREAM Triad benchmark (part of the HPC Challenge suite) and in terms of the number of memory channels available per socket. Note, these values are for the CPU systems only.

In [4]:
# Per-core memory bandwidth from StarSTREAM Triad (GB/s)
peak_bw = {}
peak_bw['ARCHER'] = 3.036
peak_bw['Peta4-Skylake'] = 4.508
peak_bw['Cirrus'] = 2.718
peak_bw['Isambard'] = 3.461
peak_bw['Tesseract'] = 5.181
# Per-socket memory channels
channels = {}
channels['ARCHER'] = 4
channels['Peta4-Skylake'] = 6
channels['Cirrus'] = 4
channels['Isambard'] = 8
channels['Tesseract'] = 6

## GROMACS: 1400k Atom Benchmark

Details of the 1400k benchmark can be found in this repository at:  https://github.com/hpc-uk/archer-benchmarks/tree/master/apps/GROMACS

Performance is measured in 'ns/day'. This is calculated by the GROMACS software itself and is read directly from the GROMACS output.

In [5]:
from appanalysis import gromacs

In [6]:
systems = ['ARCHER','Peta4-Skylake','Cirrus','Isambard','Tesseract','Wilkes2-GPU','JADE']

### Best performance per platform comparison

This section compares the best performaning configuration on a single node of each platform.

In [7]:
perf = {}
notes = {}
perf['ARCHER'] = gromacs.getperf('../apps/GROMACS/1400k-atoms/results/ARCHER/benchmark_1nodes24tasks2threads_201810051612.log')
notes['ARCHER'] = '(24 tasks, 2 threads)'
perf['Peta4-Skylake'] = gromacs.getperf('../apps/GROMACS/1400k-atoms/results/CSD3-Skylake/benchmark_1nodes32tasks1threads_201810030927.log')
notes['Peta4-Skylake'] = '(32 tasks, 1 thread)'
perf['Cirrus'] = gromacs.getperf('../apps/GROMACS/1400k-atoms/results/Cirrus/benchmark_1nodes36tasks2threads_201810022015.log')
notes['Cirrus'] = '(36 tasks, 2 threads)'
perf['Isambard'] = gromacs.getperf('../apps/GROMACS/1400k-atoms/results/Isambard/benchmark_1nodes128tasks2threads_201808201249.log')
notes['Isambard'] = '(128 tasks, 2 threads)'
perf['Tesseract'] = gromacs.getperf('../apps/GROMACS/1400k-atoms/results/Tesseract/benchmark_1nodes2threads_201810080945.log')
notes['Tesseract'] = '(24 tasks, 2 threads)'
perf['Wilkes2-GPU'] = gromacs.getperf('../apps/GROMACS/1400k-atoms/results/CSD3-GPU/benchmark_1nodes4rankspn4gpus_201808240945.log')
notes['Wilkes2-GPU'] = '(4 MPI tasks, 3 OMP per task, 4 GPU)'
# JADE performance result is taken from the runs by HEC BioSim
perf['JADE'] = 1.647
notes['JADE'] = '(5 core, 1 GPU), http://www.hecbiosim.ac.uk/jade-benchmarks'

In [8]:
formath = "{:>15s} {:>15s} {:>15s}"
formatp = "{:>15s} {:>15.3f} {:>15.3f} {:s}"
print("Performance improvement relative to ARCHER:\n")
print(formath.format('System', 'Perf. (ns/day)', 'Improvement'))
print(formath.format('======', '==============', '==========='))
gmx_perf_a = []
fp_perf_a = []
gmx_bw_a = []
peak_bw_a = []
channels_a = []
for system in systems:
    tperf = perf.get(system,0.0)
    print(formatp.format(system, tperf, tperf/perf['ARCHER'], notes.get(system, '')))
    if system in peak_fp:
        gmx_perf_a.append(perf.get(system,0.0))
        fp_perf_a.append(peak_fp.get(system,0.0))
    if system in peak_bw:
        gmx_bw_a.append(perf.get(system,0.0))
        peak_bw_a.append(peak_bw.get(system,0.0))
        channels_a.append(channels.get(system,0.0))
    

Performance improvement relative to ARCHER:

         System  Perf. (ns/day)     Improvement
         ======  ==============     ===========
         ARCHER           1.216           1.000 (24 tasks, 2 threads)
  Peta4-Skylake           2.082           1.712 (32 tasks, 1 thread)
         Cirrus           1.699           1.397 (36 tasks, 2 threads)
       Isambard           1.471           1.210 (128 tasks, 2 threads)
      Tesseract           1.323           1.088 (24 tasks, 2 threads)
    Wilkes2-GPU           2.744           2.257 (4 MPI tasks, 3 OMP per task, 4 GPU)
           JADE           1.647           1.354 (5 core, 1 GPU), http://www.hecbiosim.ac.uk/jade-benchmarks


### Correlation to peak floating point performance

Below we compute the correlation coefficients for the GROMACS benchmark performance compared to peak floating point performance. We perform two correlation tests: a Pearson correlation test to measure the correlation between numerical floating point performance and GROMACS performance; and a Spearman rank-order correlation test to measure the correlation between the order of floating point performance and GROMACS performance.

In [9]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(gmx_perf_a, fp_perf_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(gmx_perf_a, fp_perf_a)[0]))

Pearson correlation coefficient =  0.881
Spearman rank-order correlation coefficient =  0.893


Both the Pearson and Spearman rank-order correlation coefficients are high indicating that the GROMACS benchmark performance is strongly correlated to single preciaion floating point performance of the compute resources used.

### Correlation to peak memory performance

Below we compute the correlation coefficients for the GROMACS benchmark performance compared to peak floating point performance. We perform two correlation tests: a Pearson correlation test to measure the correlation between numerical floating point performance and GROMACS performance; and a Spearman rank-order correlation test to measure the correlation between the order of floating point performance and GROMACS performance.

In [10]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(gmx_bw_a, peak_bw_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(gmx_bw_a, peak_bw_a)[0]))

Pearson correlation coefficient =  0.130
Spearman rank-order correlation coefficient =  0.000


In [11]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(gmx_bw_a, channels_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(gmx_bw_a, channels_a)[0]))

Pearson correlation coefficient =  0.099
Spearman rank-order correlation coefficient =  0.211


This analysis confirms that there is no obvious correlation between the GROMACS benchmark performance and the memory performance (in terms of bandwidth or number of memory channels). This is as expected as, for this benchmark, we would not expect GROMACS to be dependent on on memory performance.

### Performance comparison matrix

In [12]:
print("{:13s}".format(''),end='')
for jsystem in systems:
    print("{:>14s}".format(jsystem), end='')
print()
for isystem in systems:
    print("{:13s}".format(isystem), end='')
    for jsystem in systems:
        print("{:14.3f}".format(perf[isystem]/perf[jsystem]), end='')
    print()

                     ARCHER Peta4-Skylake        Cirrus      Isambard     Tesseract   Wilkes2-GPU          JADE
ARCHER                1.000         0.584         0.716         0.827         0.919         0.443         0.738
Peta4-Skylake         1.712         1.000         1.225         1.415         1.574         0.759         1.264
Cirrus                1.397         0.816         1.000         1.155         1.284         0.619         1.032
Isambard              1.210         0.707         0.866         1.000         1.112         0.536         0.893
Tesseract             1.088         0.635         0.779         0.899         1.000         0.482         0.803
Wilkes2-GPU           2.257         1.318         1.615         1.865         2.074         1.000         1.666
JADE                  1.354         0.791         0.969         1.120         1.245         0.600         1.000


## OpenSBLI: Taylor-Green Vortex 512^3 benchmark

Details of the Taylor-Green Vortex 512^3 benchmark can be found in this repository at: https://github.com/hpc-uk/archer-benchmarks/tree/master/apps/OpenSBLI

Performance is measured in 'interations/s'. The total runtime and number of iterations are read directly from the OpenSBLI ouptut and these are used to compute the number of iterations per second.

In [13]:
from appanalysis import osbli

In [14]:
osbli_systems = ['ARCHER','Peta4-Skylake','Cirrus','Isambard','Tesseract']

### Best performance per platform comparison

This section compares the best performaning configuration on a single node of each platform.

In [15]:
osbli_perf = {}
osbli_perf['ARCHER'] = 1.0 / osbli.gettiming('../apps/OpenSBLI/TGV512ss/results/ARCHER/output_1nodes_201808020923.txt')
osbli_perf['Peta4-Skylake'] = 1.0 / osbli.gettiming('../apps/OpenSBLI/TGV512ss/results/CSD3-Skylake/output_1nodes_201812131243.txt')
osbli_perf['Cirrus'] = 1.0 / osbli.gettiming('../apps/OpenSBLI/TGV512ss/results/Cirrus/output_1nodes_201812201536.txt')
osbli_perf['Isambard'] = 1.0 / osbli.gettiming('../apps/OpenSBLI/TGV512ss/results/Isambard/output_1nodes_201808020732.txt')
osbli_perf['Tesseract'] = 1.0 / osbli.gettiming('../apps/OpenSBLI/TGV512ss/results/Tesseract/output_1nodes_201812171437.txt')

In [16]:
formath = "{:>15s} {:>15s} {:>15s}"
formatp = "{:>15s} {:>15.3f} {:>15.3f}"
print("Performance improvement relative to ARCHER:\n")
print(formath.format('System', 'Perf. (iter/s)', 'Improvement'))
print(formath.format('======', '==============', '==========='))
aperf = osbli_perf.get('ARCHER',0.0)
osbli_perf_a = []
fp_perf_a = []
osbli_bw_a = []
peak_bw_a = []
channels_a = []
for system in osbli_systems:
    tperf = osbli_perf.get(system,0.0)
    print(formatp.format(system, tperf, tperf/aperf))
    if system in peak_fp:
        osbli_perf_a.append(osbli_perf.get(system,0.0))
        fp_perf_a.append(peak_fp.get(system,0.0))
    if system in peak_bw:
        osbli_bw_a.append(osbli_perf.get(system,0.0))
        peak_bw_a.append(peak_bw.get(system,0.0))
        channels_a.append(channels.get(system,0.0))

Performance improvement relative to ARCHER:

         System  Perf. (iter/s)     Improvement
         ======  ==============     ===========
         ARCHER           0.100           1.000
  Peta4-Skylake           0.170           1.700
         Cirrus           0.130           1.302
       Isambard           0.178           1.777
      Tesseract           0.097           0.971


### Correlation to peak floating point performance

Below we compute the correlation coefficients for the GROMACS benchmark performance compared to peak floating point performance. We perform two correlation tests: a Pearson correlation test to measure the correlation between numerical floating point performance and GROMACS performance; and a Spearman rank-order correlation test to measure the correlation between the order of floating point performance and GROMACS performance.

In [17]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(osbli_perf_a, fp_perf_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(osbli_perf_a, fp_perf_a)[0]))

Pearson correlation coefficient =  0.692
Spearman rank-order correlation coefficient =  0.600


This indicates that there is no real correlation between floating point performance and OpenSBLI benchmark performance.

### Correlation to peak memory performance

Below we compute the correlation coefficients for the OpenSBLI benchmark performance compared to memory performance, both in terms of peak bandwdth and in terms of the number of memory channels. As for the floating point correlation analysis, we perform two correlation tests: a Pearson correlation test and a Spearman rank-order correlation test.

In [30]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(osbli_bw_a, peak_bw_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(osbli_bw_a, peak_bw_a)[0]))

Pearson correlation coefficient =  -0.065
Spearman rank-order correlation coefficient =  -0.200


In [31]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(osbli_bw_a, channels_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(osbli_bw_a, channels_a)[0]))

Pearson correlation coefficient =  0.651
Spearman rank-order correlation coefficient =  0.527


These results indicate that there is no real correlation between floating point performance and OpenSBLI benchmark performance.

Looking at the performance data by eye compared to the number of memory channels available on the different processors it appears that if the Tesseract performance is ignored, there is good correlation. We can test this:

In [32]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(osbli_bw_a[:-1], channels_a[:-1])[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(osbli_bw_a[:-1], channels_a[:-1])[0]))

Pearson correlation coefficient =  0.884
Spearman rank-order correlation coefficient =  0.949


This test confirms that the Tesseract performance does not fit the pattern of performance correlated to number of memory channels. We would expect the performance of a CFD application such as OpenSBLI to be dependent on random memory access performance which, in turn, is mediated by the number of memory channels available and so this result fits this expectation (with the exception of the Tesseract performance).

### Performance comparison matrix

In [33]:
print("{:13s}".format(''),end='')
for jsystem in osbli_systems:
    print("{:>14s}".format(jsystem), end='')
print()
for isystem in osbli_systems:
    print("{:13s}".format(isystem), end='')
    iperf = osbli_perf[isystem]
    for jsystem in osbli_systems:
        jperf = osbli_perf[jsystem]
        print("{:14.3f}".format(iperf/jperf), end='')
    print()

                     ARCHER Peta4-Skylake        Cirrus      Isambard     Tesseract
ARCHER                1.000         0.588         0.768         0.563         1.030
Peta4-Skylake         1.700         1.000         1.306         0.957         1.751
Cirrus                1.302         0.766         1.000         0.733         1.341
Isambard              1.777         1.045         1.365         1.000         1.830
Tesseract             0.971         0.571         0.746         0.547         1.000


## CASTEP: Al Slab benchmark

Details of the Al Slab benchmark can be found in this repository at:  https://github.com/hpc-uk/archer-benchmarks/blob/master/apps/CASTEP/

Performance is measured in 'mean SCF cycles per second'. This is calculated from the CASTEP output files by computing the SCF cycle times, removing the minimum and maximum value and then computing the mean of the remaining values.

In [34]:
from appanalysis import castep

In [35]:
castep_systems = ['ARCHER','Peta4-Skylake','Cirrus','Isambard','Tesseract']

### Best performance per platform comparison

This section compares the best performaning configuration on a single node of each platform.

In [36]:
castep_perf = {}
castep_perf['ARCHER'] = 1.0 / castep.getmeancycle('../apps/CASTEP/al3x3/results/ARCHER/al3x3.castep.1nodes')
castep_perf['Peta4-Skylake'] = 1.0 / castep.getmeancycle('../apps/CASTEP/al3x3/results/CSD3-Skylake/al3x3.castep.1nodes')
castep_perf['Cirrus'] = 1.0 / castep.getmeancycle('../apps/CASTEP/al3x3/results/Cirrus/17.21_gcc620_impi17/al3x3.castep.1nodes')
castep_perf['Isambard'] = 1.0 / castep.getmeancycle('../apps/CASTEP/al3x3/results/Isambard/al3x3.castep.1nodes_201806130634')
castep_perf['Tesseract'] = 1.0 / castep.getmeancycle('../apps/CASTEP/al3x3/results/Tesseract/al3x3_1nodes_201808071417.castep')

In [37]:
formath = "{:>15s} {:>15s} {:>15s}"
formatp = "{:>15s} {:>15.5f} {:>15.3f}"
print("Performance improvement relative to ARCHER:\n")
print(formath.format('System', 'Perf. (scf/s)', 'Improvement'))
print(formath.format('======', '==============', '==========='))
aperf = castep_perf.get('ARCHER',0.0)
castep_perf_a = []
fp_perf_a = []
castep_bw_a = []
peak_bw_a = []
channels_a = []
for system in castep_systems:
    tperf = castep_perf.get(system,0.0)
    print(formatp.format(system, tperf, tperf/aperf))
    if system in peak_fp:
        castep_perf_a.append(castep_perf.get(system,0.0))
        fp_perf_a.append(peak_fp.get(system,0.0))
    if system in peak_bw:
        castep_bw_a.append(castep_perf.get(system,0.0))
        peak_bw_a.append(peak_bw.get(system,0.0))
        channels_a.append(channels.get(system,0.0))

Performance improvement relative to ARCHER:

         System   Perf. (scf/s)     Improvement
         ======  ==============     ===========
         ARCHER         0.00543           1.000
  Peta4-Skylake         0.01641           3.023
         Cirrus         0.01109           2.043
       Isambard         0.00691           1.273
      Tesseract         0.00728           1.341


### Correlation to peak floating point performance

Below we compute the correlation coefficients for the CASTEP benchmark performance compared to peak floating point performance. We perform two correlation tests: a Pearson correlation test to measure the correlation between numerical floating point performance and CASTEP performance; and a Spearman rank-order correlation test to measure the correlation between the order of floating point performance and CASTEP performance.

In [38]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(castep_perf_a, fp_perf_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(castep_perf_a, fp_perf_a)[0]))

Pearson correlation coefficient =  0.953
Spearman rank-order correlation coefficient =  0.900


These analyses suggest a strong correlation between the peak floating point performance of the compute nodes and the performance of the CASTEP benchmark. The correlation for this CASTEP benchmark is stronger than the correlation for GROMACS; which is traditionally though of a pure floating point application. This is probably due to the fact that, on a single node, most of the time for this CASTEP benchmark is spent in LAPACK numerical routines which are well-optimed to exploit the maximum floating point performance from the processors.

### Correlation to peak memory performance

Below we compute the correlation coefficients for the CASTEP benchmark performance compared to memory performance, both in terms of peak bandwdth and in terms of the number of memory channels. As for the floating point correlation analysis, we perform two correlation tests: a Pearson correlation test and a Spearman rank-order correlation test.

In [39]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(castep_bw_a, peak_bw_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(castep_bw_a, peak_bw_a)[0]))

Pearson correlation coefficient =  0.223
Spearman rank-order correlation coefficient =  0.200


In [40]:
print('Pearson correlation coefficient = ', '{:.3f}'.format(stats.pearsonr(castep_bw_a, channels_a)[0]))
print('Spearman rank-order correlation coefficient = ', '{:.3f}'.format(stats.spearmanr(castep_bw_a, channels_a)[0]))

Pearson correlation coefficient =  -0.013
Spearman rank-order correlation coefficient =  0.053


The results for memory correlation suggest there is no correlation between node memory performance and the perfomance of this CASTEP benchmark on a single node.

### Performance comparison matrix

In [41]:
print("{:13s}".format(''),end='')
for jsystem in castep_systems:
    print("{:>14s}".format(jsystem), end='')
print()
for isystem in castep_systems:
    print("{:13s}".format(isystem), end='')
    iperf = castep_perf[isystem]
    for jsystem in castep_systems:
        jperf = castep_perf[jsystem]
        print("{:14.3f}".format(iperf/jperf), end='')
    print()

                     ARCHER Peta4-Skylake        Cirrus      Isambard     Tesseract
ARCHER                1.000         0.331         0.489         0.786         0.746
Peta4-Skylake         3.023         1.000         1.479         2.375         2.254
Cirrus                2.043         0.676         1.000         1.605         1.524
Isambard              1.273         0.421         0.623         1.000         0.949
Tesseract             1.341         0.444         0.656         1.054         1.000
